In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
#from google.colab import files
# test
#uploaded = files.upload() #per 36 minutes, per 100 poss, player award shares, player per game, player totals, advanced, all star selections, end of season teams (voting), team summaries

In [8]:
#cite chat gpt later
csv_files = [
    'Per 100 Poss.csv', 'Per 36 Minutes.csv', 'Player Award Shares.csv',
    'Player Per Game.csv',
    'Player Totals.csv','Advanced.csv', 'All-Star Selections.csv', 'End of Season Teams (Voting).csv',
]

# Dictionary to store DataFrames
dataframes = {}

# Read each CSV file and store it in the dictionary
for file in csv_files:
    # Construct the full file path assuming the CSV files are in the current working directory
    file_path = file

    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Store the DataFrame in the dictionary with the file name as the key
    dataframes[file] = df

# Now you can access each DataFrame using its file name as the key
# For example, to access the 'Per 100 Poss.csv' DataFrame:
per_100_poss_df = dataframes['Per 100 Poss.csv']

# Display the first few rows of the DataFrame
dataframes['Advanced.csv']

FileNotFoundError: [Errno 2] No such file or directory: 'Per 100 Poss.csv'

In [9]:
for i in csv_files: #This double for loop filters all of our data to only seasons past the year 2000
  for row in dataframes[i]:
    dataframes[i] = dataframes[i][dataframes[i]['season'] >= 2000]
dataframes['Player Award Shares.csv']

KeyError: 'Per 100 Poss.csv'

In [ ]:
#This cell filters the player award shares dataframe to only display the players associated with the mvp award (we don't care about the other awards)
a = dataframes['Player Award Shares.csv'][(dataframes['Player Award Shares.csv']['award'] == 'nba mvp')]

In [ ]:
"""This is a large cell that does all of our merges. Some of the merges are outer and some are inner because the outer merges
created too many unneccesary columns and duplicate player rows"""
merged_df = dataframes['All-Star Selections.csv']
merged_df = merged_df.merge(a, how='outer', on = ['player', 'season'])
merged_df['made_asg'] = True
merged_df['winner'] = merged_df['winner'].fillna(False)
merged_df = merged_df.fillna(0)
merged_df = merged_df.merge(dataframes['Advanced.csv'], how='outer', on = ['player', 'season', 'seas_id'])
merged_df['made_asg'] = merged_df['made_asg'].fillna(False)
merged_df['winner'] = merged_df['winner'].fillna(False) #merged_df was used to experiment on different decision trees
merged_df = merged_df.fillna(0)
merged_df3 = merged_df.merge(dataframes['Player Per Game.csv'], how = 'inner', on = ['player', 'season', 'seas_id']) #merged_df3 is used for the random forest and stats tests
merged_df3 = merged_df3.merge(dataframes['Per 100 Poss.csv'], how = 'inner', on = ['player', 'season', 'seas_id'])
merged_df3 = merged_df3.merge(dataframes['Per 36 Minutes.csv'], how = 'inner', on = ['player', 'season', 'seas_id'])
merged_df3 = merged_df3.merge(dataframes['Player Totals.csv'], how = 'inner', on = ['player', 'season', 'seas_id'])


In [ ]:
merged_df3 #Note that whenever a player was not mentioned in NBA MVP voting, they had 0s in the MVP columns because of .fillna()

In [ ]:
merged_df2 = pd.merge(dataframes['End of Season Teams (Voting).csv'], a, on='seas_id', how='inner')
merged_df2 #used for first decision tree

In [ ]:
y = [int(x) for x in merged_df2['winner']] #beginning the train/test split, first identifying X and y
X = merged_df2.select_dtypes(include= ['float', 'int']).fillna(0)

In [ ]:
from sklearn.model_selection import train_test_split #Decision Tree
X_train, X_test, y_train, y_test = \
train_test_split(X,y, random_state = 110)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(criterion="entropy", random_state=110)
dtree.fit(X_train, y_train)
dtree.score(X_test, y_test)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import tree

plt.figure(figsize=(14,10))
tree.plot_tree(dtree) #this ddecision tree is implying that all our past mvps have gotten at least 51 votes in first team all NBA


In [ ]:
X.iloc[:,12] #Evidence that index 12 = first team

This next Decision Tree that we created differs from the other one because it is ran on data from merged_df, which includes much more data from other files.

In [ ]:
merged_df['made_asg'] = [int(x) for x in merged_df['made_asg']]
y2 = [int(x) for x in merged_df['winner']]
X2 = merged_df.select_dtypes(include= ['float', 'int']).fillna(0)
X2 = X2.drop(columns = ['pts_max', 'share', 'season', 'seas_id', 'player_id_x', 'player_id_y', 'first', 'pts_won'])
X2.dtypes

In [ ]:
from sklearn.model_selection import train_test_split #Decision tree 2
X2_train, X2_test, y2_train, y2_test = \
train_test_split(X2,y2, random_state = 110)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(criterion="entropy", random_state=110)
dtree.fit(X2_train, y2_train)
dtree.score(X2_test, y2_test)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import tree

plt.figure(figsize=(14,10))
tree.plot_tree(dtree)

In [ ]:
X2.iloc[:,26]

In [ ]:
merged_df3['made_asg'] = [int(x) for x in merged_df3['made_asg']] # creating X3 and y3 for the random forest
y3 = [int(x) for x in merged_df3['winner']]
X3 = merged_df3.select_dtypes(include= ['float', 'int']).fillna(0)
X3 = X3.drop(columns = ['pts_won', 'pts_max', 'share', 'season', 'seas_id', 'player_id_x', 'player_id_y', 'age_x', 'first', 'experience', 'ws_48' ])
#everything after these stats above are to help make it more accurate. keep the ones above


In [ ]:
X3

In [ ]:
X3drop = X3.filter(like = '_y').columns #filtering out duplicated data and/or irrelevant data
X3 = X3.drop(columns = X3drop)
for a in X3.columns:
  print(a)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X3_train, X3_test, y3_train, y3_test = \
train_test_split(X3,y3, random_state = 110)#random forest

In [ ]:
random_forest = RandomForestClassifier(n_estimators=200, bootstrap = False)
random_forest.fit(X3_train, y3_train)
random_forest.score(X3_test, y3_test)

In [ ]:
importances = random_forest.feature_importances_ #importances or significance of each data is stored here

In [ ]:
importances_dict = {} # puts an index that corresponds to merged_df3 column in a dictionary with the importances
for i in range(len(importances)):
  importances_dict[i] = importances[i]
importances_dict
sorted_importances = sorted(importances_dict.items(), key=lambda x: x[1], reverse=True) #chat gpt

In [ ]:
stats = [] #This cell adds the index of the merged_df3 column to each importance stat so it's easy to see
num = 0
for i in X3.columns:
  stats.append(i + ' / ' + str(num))
  num +=1
stats_per_importance = {}
for i in range(len(stats)):
  stats_per_importance[stats[i]] = importances[i]
stats_per_importance

In [ ]:
sorted_stats = dict(sorted(stats_per_importance.items(), key=lambda item: item[1], reverse = True)) #This cell sorts the previous dictionary by highest value
sorted_stats

In [ ]:
def scale_column(column): #making a function to use .apply on, got from chat gpt
    max_value = column.max() #This cell scales the X3 dataframe to make the random forest more accurate
    scaled_column = column / max_value
    return scaled_column

scaled_X3 = X3.apply(scale_column, axis=0)

In [ ]:
X4_train, X4_test, y4_train, y4_test = \
train_test_split(scaled_X3,y3, random_state = 110)#4th train/test split - this one is for the scaled random forest

random_forest2 = RandomForestClassifier(n_estimators=300, bootstrap = True)
random_forest2.fit(X4_train, y4_train)
random_forest2.score(X4_test, y4_test)

In [ ]:
importances2 = random_forest2.feature_importances_

In [ ]:
importances_dict2 = {} #doing the same as we did above with the importances dictionary where there is an index and the column name as well so it is readable
for i in range(len(importances)):
  importances_dict2[i] = importances2[i]
importances_dict2
sorted_importances2 = sorted(importances_dict2.items(), key=lambda x: x[1], reverse=True)

In [ ]:
stats2 = []
num2 = 0
for i in scaled_X3.columns:
  stats2.append(i + ' / ' + str(num2))
  num2 +=1
stats_per_importance2 = {}
for i in range(len(stats2)):
  stats_per_importance2[stats2[i]] = importances2[i]

In [ ]:
sorted_stats2 = dict(sorted(stats_per_importance2.items(), key=lambda item: item[1], reverse = True))
sorted_stats2

In [ ]:
def calculate_sum_of_products(row, array): #This function helps us calculate each players' individual 'mvp scores'
    return np.sum(np.array(row) * np.array(array))

In [ ]:
mvp_scores = [] #creates the mvp scores list
for i in range(len(X3)):
  a = calculate_sum_of_products(X3.iloc[i,:], importances2)
  mvp_scores.append(a)


In [ ]:
final_df = merged_df3 #makes a new column in the merged_df3 that is mvp scores
final_df['mvp_scores'] = mvp_scores
seasons = np.arange(2000,2024,1)
df_by_season = {} #This is gonna gelp show the highest mvp score in each year
for i in seasons:
  a = merged_df3[merged_df3['season'] == i]
  df_by_season[i] = a


In [ ]:
predicted_mvp_per_year = [] #finds the name of the person with the highest mvp score for each year starting at 2000
for i in seasons:
  max_index = df_by_season[i]['mvp_scores'].idxmax()
  row = df_by_season[i].loc[max_index]
  predicted_mvp_per_year.append(row['player'])


In [ ]:
predicted_mvp_per_year #our predicted mvps from 2000-2023

In [ ]:
real_mvps_pre = dataframes['Player Award Shares.csv'][dataframes['Player Award Shares.csv']['award'] == 'nba mvp']
mvp_names = [] #This cell displats the real mvp results, will use to see the accuracy of our test

for index, row in real_mvps_pre.iterrows():
    if row['winner']:
        mvp_names.append(row['player'])
mvp_names.reverse()


In [ ]:
mvp_names #The real mvps (our predictions are quite different)

In [ ]:
percent_accuracy = 0 #Testing how many we got right/wrong
t_or_f_accuracy = []
for i in range(len(mvp_names)):
  if mvp_names[i] == predicted_mvp_per_year[i]:
    percent_accuracy += 1
    t_or_f_accuracy.append(True)
  else:
    t_or_f_accuracy.append(False)
t_and_f = [percent_accuracy, 23 - percent_accuracy]
t_and_f

In [ ]:
t_or_f_accuracy

In [ ]:
import scipy.stats
true_list = [True for i in range(23)] #This cell is doing a t-test between the average rates of True mvp predictions from our model and the real model (which is 100% true)
z_stat, p_value = scipy.stats.ttest_ind(t_or_f_accuracy, true_list)
p_value #This p-value means that we reject the null that our model aligns with the real model, and our model is not 100% accurate. (obviously)

In [ ]:
corr_df = merged_df3.corr(numeric_only = True) #This is finding the correlation between each statistic

In [ ]:
corr_df[['share']].sort_values(by='share', ascending=False).head(10) #showing the 10 highest correlations with the MVP voting share
#Note that many of these stata are the ones we deleted for the random forests and therefore are irrelevant

In [ ]:
corr_df[['share']].sort_values(by='share', ascending=False).iloc[8:18] #The highest correlations within normal stats and share which is the share of the mvp voting pool are also the stats that were voted most important by the random forest

**Visualizations**

In [ ]:
#Make the Data usable for the graphs
csv_files1 = ['Player Award Shares.csv', 'Team Summaries.csv']
dataframes1 = {}
for file in csv_files1:
    file_path1 = file
    df1 = pd.read_csv(file_path1)
    dataframes1[file] = df1
dataframes1['Player Award Shares.csv'].head()

In [ ]:
Graph2 = dataframes1['Player Award Shares.csv']
for i in csv_files1:
  for row in dataframes1[i]:
    dataframes1[i] = dataframes1[i][dataframes1[i]['season'] >= 1977] #first year of ABA and NBA merger so let's use this data only
for i in csv_files1:
  for row in dataframes1[i]:
    dataframes1[i] = dataframes1[i][dataframes1[i]['season'] >= 1977]
Graph2 = Graph2[~Graph2['award'].str.contains('smoy|dpoy|mip|nba roy')] #we're only analyzing nba mvps so we delete all other data that are not nba mvps
Graph2 = Graph2[~(Graph2['winner']==False)] #again we only want mvp winners so we delete all other data
Graph2


In [ ]:
#using data from url: https://hoop-social.com/nba-team-market-size-rankings/ we add another column to our data to include market size alogn with the teams
def merge_column(row):
  if 'ATL'in row['tm'] or 'BRK'in row['tm'] or 'BOS' in row['tm'] or 'CHI' in row['tm'] or 'DAL' in row['tm'] or 'GSW' in row['tm'] or 'HOU' in row['tm'] or 'LAC' in row['tm'] or 'LAL' in row['tm'] or 'NYK' in row['tm'] or 'PHI' in row['tm'] or 'PHO' in row['tm'] or 'TOR' in row['tm'] or 'WAS' in row['tm']:
    return 'Large'
  elif 'CLE'in row['tm'] or 'DEN'in row['tm'] or 'DET' in row['tm'] or 'MIA' in row['tm'] or 'MIN' in row['tm'] or 'ORL' in row['tm']:
    return 'Medium'
  elif 'IND'in row['tm'] or 'MEM'in row['tm'] or 'MIL' in row['tm'] or 'NOP' in row['tm'] or 'OKC' in row['tm'] or 'POR' in row['tm'] or 'SAC' in row['tm'] or 'SAS' in row['tm'] or 'UTA' in row['tm']:
    return 'Small'
Graph2['market size'] = Graph2.apply(merge_column, axis=1) #merge function to make it one table
Graph2


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
market_counts = Graph2['market size'].value_counts() #this variable is our frequency (number of times) large, medium, small appear in the column
labels = ['Large', 'Medium', 'Small']
colors = [ 'firebrick', 'cadetblue', 'linen']
#graph details and features
fig, ax = plt.subplots(figsize=(16, 9))
bars = ax.bar(labels, market_counts, color = colors)
ax.xaxis.set_ticks_position('none')
ax.yaxis.set_ticks_position('none')
ax.xaxis.set_tick_params(pad=5)
ax.yaxis.set_tick_params(pad=10)
plt.xlabel("Market Size")
plt.ylabel("Number of MVPs")
bar_color = bars[0].get_facecolor()
for bar in bars:
  ax.text(
      bar.get_x() + bar.get_width() / 2,
      bar.get_height() + 0.3,
      round(bar.get_height(), 1),
      horizontalalignment='center',
      weight='bold'
  )
ax.set_title('Number of MVP Players from Market Size',
             loc='Center', )
ax.set_axisbelow(True)
ax.yaxis.grid(True, color='#EEEEEE')
ax.xaxis.grid(False)
plt.show()

In [ ]:
for i in csv_files1:
  for row in dataframes1[i]:
    dataframes1[i] = dataframes1[i][dataframes1[i]['season'] >= 1976] #first year of ABA and NBA merger is 1977 so we start our data from there
#this function will take a year after 1976 as an input and display a graph of the wins that each team got in the regular season with a highlighted bar of the MVP's team
def Team_Wins_In_Season (year):
  from matplotlib import pyplot as plt
  Graph3 = dataframes1['Team Summaries.csv']
  bad_str = ['League Average']
  Graph3 = Graph3[~Graph3['team'].isin(bad_str)] #We dont want to include the League Average rows so we remove them from the dataset
  Seasonstats = Graph3.loc[(Graph3['season'] == year)] #with the input variable we only extract the data from that season
  mvp_tm = Graph2.loc[(Graph2['season'] == year), 'tm'].values[0] #returns string of team abbreviation of the MVP for that year
  mvp_team = Seasonstats.loc[(Graph3['abbreviation'] == mvp_tm), 'team'].values[0] #in order to connect the mvp_tm variable with Seasonstats we set them equal to each other using the abbreviation column in Seasonstats
  wins_tm = Seasonstats[['team', 'w']] #returns Dataframe with teams and number of wins
  #graph features and detaisl
  fig, ax = plt.subplots(figsize=(16, 9))
  colors = ['red' if category == mvp_tm else 'gray' for category in Seasonstats['abbreviation']] #highlights the team where the MVP is from
  ax.barh(Seasonstats['abbreviation'], Seasonstats['w'], color=colors)
  ax.xaxis.set_ticks_position('none')
  ax.yaxis.set_ticks_position('none')
  ax.xaxis.set_tick_params(pad=5)
  ax.yaxis.set_tick_params(pad=10)
  ax.invert_yaxis()
  for i in ax.patches:
    plt.text(i.get_width()+0.2, i.get_y()+0.5,
            str(round((i.get_width()), 2)), fontsize=10, fontweight='bold', color='black')
  ax.set_title("Team Wins in " + str(year) + " with the MVP from: " + mvp_team + " (" + mvp_tm + ")",
             loc='center', fontsize=18)
  plt.show()
  return

In [ ]:
Team_Wins_In_Season(2000)

In [ ]:
sns.relplot(data=merged_df3, x="pts", y="mvp_scores", hue="winner", size = 'winner', sizes = (200,10))
plt.title("The Relationship Between Points and MVP Score", fontsize=16)
#The graph shows the relationship between total pts scored over the season and our mvp_scores, as well as enlarging the true MVP winners
#Note the true MVP winners have higher mvp scores for the most part, and they also score a lot of points

In [ ]:
ppg = pd.read_csv('Player Per Game.csv')
ast = pd.read_csv('All-Star Selections.csv')

#per 36 minutes, per 100 poss, player award shares, player per game, player totals, advanced, all star selections, end of season teams (voting)

import sklearn.linear_model as lm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
## take both csv files and merge them. Then create a histogram which has the points per game of every nba player. After that
#take the all star names and change the color of the bars containing all stars



# extract the necessary data and dmerge
ppg_2020 = ppg[ppg['season']== 2020]
points = ppg_2020['pts_per_game']
all_stars_2020 = ast[ast['season']==2020]


result = ppg_2020[['player', 'pts_per_game']]
#create the histogram
bins = [ 0, 5, 10, 15, 20, 25, 30, 35]
hist_values, bins, _ = plt.hist(result['pts_per_game'], bins = bins, edgecolor = 'black', color = 'lavender')


ast_p =ast[ast['season'] == 2020]['player']
ast_i = result.index[result['player'].isin(ast_p)].tolist()
#Loop whuch implements the colors
for i,start in  enumerate(bins[:-1]):
  end =  bins[i+1]
  ast_in_bin = result[(result['pts_per_game']>= start) & (result['pts_per_game'] < end)]['player'].isin(ast_p)
  ast_i_in_bin = ast_in_bin.index[ast_in_bin].tolist()
  for j in ast_i_in_bin:
     plt.bar(bins[i], hist_values[i], width=bins[i + 1] - bins[i], align='edge', color='lightcoral', edgecolor='black') #This coral hightlights if there is an all star in that category

#draws the histogram
for i, txt in enumerate(hist_values):
  plt.annotate(int(txt), (bins[i], hist_values[i]), ha = 'center', va = 'bottom')
plt.xlabel("Points Per Game")
plt.ylabel("Number Of Players")
plt.title('The Distribution of ppg of NBA players from 2020')

plt.xticks(bins[:-1])


plt.show()

In [ ]:
data = final_df[['mvp_scores']] #This is a histogram that shows the dsitribution of our computed mvp_scores
sns.histplot(data, kde=False, color='skyblue', bins=30)
plt.xlim(0)
plt.title('Distribution of mvp_scores of players from 2000-2023')
plt.xlabel('mvp_scores')